In [ ]:
from py_markdown_table.markdown_table import markdown_table
import gitlab

#create the gitlab object by entering a personal access token
input_token = input("Enter access token: ")
gl = gitlab.Gitlab(private_token=input_token)
project_id = input("Enter Project ID")

#create the project object based on the project id number
project = gl.projects.get(project_id)
#get the members of the project and add them to an array
members = project.members.list()
names = [member.name for member in members]
names.remove(names[0])


#Method to check if the comment has a date in it for parsing purposes
def contains_date(body):
    if '-' in body:
        return True
    else:
        return False


#Splitting the comments body and parsing it based on the inputted lengths   
def splitting_body(body: str, length1: int, length2: int, pos: bool):
    total = 0
    body_split = body.split(" ")
    if len(body_split) == length1:
        hour = int(body_split[1].replace('h', ''))
        minute = int(body_split[2].replace('m', ''))
        minute_prop = float(minute / 60)
        total = hour + minute_prop
    elif len(body_split) == length2:
        if 'h' in body_split[1]:
            total = int(body_split[1].replace('h', ''))
        else:
            total = float(int(body_split[1].replace('m', '')) / 60)
    if pos:
        return total
    else:
        return -total


#Inputting the body of a comment to be parsed down into the time spent by that user
def parse_body(body):
    total = 0
    #Positive time to be added to the total (True)
    if 'time spent' in body:
        if contains_date(body):
            total = splitting_body(body, 8, 7, True)
        else:
            body_split = body.split(" ")
            total = splitting_body(body_split, 6, 5, True)
        return total
    #Negative times to be removed from the total (False)
    elif 'spent time' in body:
        if contains_date(body):
            total = splitting_body(body, 8, 7, False)
        else:
            total = splitting_body(body, 6, 5, False)
        return total
    #If the comment body is not a spending time comment, return 0
    else:
        return total


#Test the functionality of parse_body on a single issue's comments
comments_ceremonies_1 = project.issues.get(96).notes.list(get_all=True)


#Takes in the list of comments per each PBI 
#Uses parse_body to get the time per person per PBI
def time_by_pbi(comment_list):
    time_by_user = []
    for x in names:
        time_by_user.append([x, 0])
    for comment in comment_list:
        body = comment.body
        author = comment.author["name"]
        time = parse_body(body)
        for user in time_by_user:
            if user[0] == author:
                user[1] += time
    return time_by_user


#Get only the issues included in the '24f2' milestone
sprint2_issues = project.issues.list(milestone='24f2')

#Create and fill an array of dictionaries where the keys are an empty string 
#and the value is the title of each issue
time_spent = []
for i in range(len(sprint2_issues)):
    time_spent.append({"": sprint2_issues[i].title})

#Create a new array for the total time spent by a user
totals = []
#Fill the array with 0 for the amount of users in the project
for j in range(len(names)):
    totals.append(0)
#Iterate through the list of issues for the second sprint issues 
for issue in sprint2_issues:
    #get the comment list for each issue and put through the time_by_pbi function
    comments = issue.notes.list(get_all=True)
    temp_array = time_by_pbi(comments)
    #Add to the total amount of time per user 
    for x in range(len(temp_array)):
        totals[x] += temp_array[x][1]
    #Iterate through the time_spent array as each dictionary must be added to
    for y in range(len(time_spent)):
        #If the time_spent dictionary at index y is the pbi of the current issue
        #iterate through the array returned by time_by_pbi
        if time_spent[y][""] == issue.title:
            for z in range(len(temp_array)):
                #create a new key in the dictionary of the user's name and add the value of their time.
                time_spent[y][temp_array[z][0]] = temp_array[z][1]

#Create the totals dictionary to be added to the time_spent array
total_dict = {"": "Total"}
#Iterate through the length of totals and create a new key for each user 
#with the value being their total time
for a in range(len(totals)):
    total_dict[names[a]] = totals[a]
time_spent.append(total_dict)

sprint1_issues = project.issues.list(milestone='24f1')

time_spent_2 = []
for i in range(len(sprint1_issues)):
    time_spent_2.append({"": sprint1_issues[i].title})

totals = []
for j in range(len(names)):
    totals.append(0)
for issue in sprint1_issues:
    comments_1 = issue.notes.list(get_all=True)
    temp_array = time_by_pbi(comments_1)
    for x in range(len(temp_array)):
        totals[x] += temp_array[x][1]
    # print(temp_array)
    # print(issue.title)
    for y in range(len(time_spent_2)):
        if time_spent_2[y][""] == issue.title:
            # print("here")
            for z in range(len(temp_array)):
                time_spent_2[y][temp_array[z][0]] = temp_array[z][1]
            # print(time_spent[y])
total_dict = {"": "Total"}

for a in range(len(totals)):
    total_dict[names[a]] = totals[a]
time_spent_2.append(total_dict)
sprint_1_markdown = markdown_table(time_spent_2).set_params(float_rounding=2, padding_width=2, quote=False, row_sep='markdown').get_markdown()
sprint_2_markdown = markdown_table(time_spent).set_params(float_rounding=2, padding_width=2, quote=False, row_sep='markdown').get_markdown()


